In [ ]:
import os
import sys
import numpy as np
import pandas as pd
from scipy import stats

import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import matplotlib.markers as mmark
plt.rcParams.update({'figure.max_open_warning': 0})
plt.rcParams.update({'font.size': 16})

from FunctionsDLCA import *

from contextlib import contextmanager
import importlib

In [ ]:
@contextmanager
def blockPrint():
    old_stdout = sys.stdout
    sys.stdout = None
    try:
        yield

    finally:
        sys.stdout = old_stdout

In [ ]:
# Softening Function
def soften(arr, steps):
    if steps == 0:
        steps = 1
    new_arr = []
    for i in range(0, len(arr), steps):
        try:
            new_arr.append(np.sum(arr[i:i+steps]) / steps)
        except:
            continue
    return new_arr

In [ ]:
# Globals
    # Clusters
global lat_size #INT
global radius #INT
global densities #LIST
global colors #LIST
colors = ["black", "darkslategrey", "teal", "royalblue", "midnightblue", "navy", "slateblue", "indigo"]
markers = ["o", "^", "*", "P", "x", "1"]
global path_seagate
path_seagate = "D:/ASE III Resultados/"

    # File Names
global cluster_fn
cluster_fn = "Cluster"
global frac_dim_counts_fn
frac_dim_counts_fn = "FracDimList" 
global rg2_mass_t_fn
rg2_mass_t_fn = "RgMassTime"
global mass_time_fn
mass_time_fn = "NumberTime"
global edges_fn
edges_fn = "Edge"

In [ ]:
df = pd.read_csv("Final Results/ResultsOffL.csv")

lat_size = df.lattice_size.unique()
lat_size.sort()
print(lat_size)

radius = 1

simulations = dict()
simulation_probabilities = dict()
simulation_densities = dict()
for lat in lat_size:
    simulations[lat] = sorted(list(df[df.lattice_size == lat].num_particles.unique()))
    simulation_probabilities[lat] = sorted(list(df[df.lattice_size == lat].probability.unique()))
    simulation_densities[lat] = [(particles * np.pi * (radius ** 2)) / (4 * (lat ** 2)) for particles in sorted(list(df[df.lattice_size == lat].num_particles.unique()))]

print(simulations)
print(simulation_probabilities)
print(simulation_densities)

## Fractal Dimensions

In [ ]:
# Function that counts the average number of particles per box in different divisions, calculates the size of these boxes,
# and calculates the amount of boxes that contain particles for a given division size.
def boxCount(L, particles, x, y):
    div = 4 + (4 * (particles <= 8000)) # Sets start division
    break_condition = 2 + (2 * (L > 200)) # Sets ending place for amount of divisions
    box_count = [] # List containing the amount of boxes with particles
    avg_counts = [] # List containing the amount of particles average per box
    box_size = [] # List containing the size of the current boxes
    while True: # Emulation of do while loop to calculate individual vales to put into previous lists
        boxcount = 0
        division = L / div

        if division < break_condition: # Ends do while loop
            break

        for i in range(1,div+1):
            for j in range(1,div+1):
                lower_x = ((i - 1) * division)
                upper_x = (i * division)
                lower_y = ((j-1) * division)
                upper_y = (j * division)
                for xi, yi in zip(x, y):
                    if lower_x <= xi <= upper_x and lower_y <= yi <= upper_y:
                        boxcount += 1
                        break # If a box contains 1 particle the code can move on to the next

        # Appends found values to the lists
        box_count.append(boxcount)
        avg_counts.append(particles/boxcount)
        box_size.append(division)

        print("{:.3f} divisions with minimum {}".format(division, break_condition))

        div *= 2

    return box_count, avg_counts, box_size

In [ ]:
box_count_lat = dict()
avg_count_lat = dict()
box_sizes_lat = dict()

for lat in lat_size:
    box_count_lat[lat] = dict()
    avg_count_lat[lat] = dict()
    box_sizes_lat[lat] = dict()
    for particles in simulations[lat]:
        box_count_lat[lat][particles] = dict()
        avg_count_lat[lat][particles] = dict()
        box_sizes_lat[lat][particles] = dict()
        
        for probability in simulation_probabilities[lat]:
            box_counts_list = []
            avg_counts_list = []
            box_sizes_list = []

            text_files = [f for f in os.listdir(path_seagate) if (cluster_fn in f) and (str(int(lat)) in f) and (str(int(particles)) in f) and (str(float(probability)) in f) and not (edges_fn in f)]
            if text_files:
                box_count_lat[lat][particles][probability] = []
                avg_count_lat[lat][particles][probability] = []
                box_sizes_lat[lat][particles][probability] = []
                print(f"Computing Fractal Dims for Lat = {lat}, Particles = {particles} and Probability = {probability}")
                for file_i in text_files:
                    cluster = np.loadtxt(path_seagate + '/' + file_i, delimiter = ',')

                    x, y, index, z = np.hsplit(cluster, cluster.shape[1])
                    x = x.flatten()
                    y = y.flatten()
                    indexes = index.flatten().astype(int)
                    largest_cluster = int(stats.mode(indexes).mode)

                    x_index = x[np.where(indexes == largest_cluster)]
                    y_index = y[np.where(indexes == largest_cluster)]
            
                    if (len(x_index)/particles) > 0.4:
                        with blockPrint():
                            box_count, avg_counts, box_size = boxCount(lat, len(x_index), x_index, y_index)
                        plt.close('all')

                        box_counts_list.append(box_count)
                        avg_counts_list.append(avg_counts)
                        box_sizes_list.append(box_size)

            if box_counts_list:
                box_count_avg_p = np.mean(box_counts_list, axis = 0)
                avg_count_avg_p = np.mean(avg_counts_list, axis = 0)
                box_sizes_avg_p = np.mean(box_sizes_list, axis = 0)
                box_count_lat[lat][particles][probability].append(box_count_avg_p)
                avg_count_lat[lat][particles][probability].append(avg_count_avg_p)
                box_sizes_lat[lat][particles][probability].append(box_sizes_avg_p)

print(box_sizes_lat)
print(box_count_lat)
print(avg_count_lat)


In [ ]:
import pickle

# box_sizes_lat_viejo = box_sizes_lat
# box_count_lat_viejo = box_count_lat
# avg_count_lat_viejo = avg_count_lat


# with open("box_sizes_lat_count.pkl", 'wb') as f:
#     pickle.dump(box_sizes_lat, f)

# with open("box_count_lat_count.pkl", 'wb') as f:
#     pickle.dump(box_count_lat, f)

# with open("avg_count_lat_count.pkl", 'wb') as f:
#     pickle.dump(avg_count_lat, f)

In [ ]:
import pickle
with open('box_sizes_lat_count.pkl', 'rb') as f:
    box_sizes_lat_file = pickle.load(f)

with open('box_count_lat_count.pkl', 'rb') as f:
    box_count_lat_file = pickle.load(f)

with open('avg_count_lat_count.pkl', 'rb') as f:
    avg_count_lat_file = pickle.load(f)

print(box_sizes_lat_file)

In [ ]:
print("Computing logarithm of Lists")
log_box_sizes_lat = dict() # log average box count list
log_box_count_lat = dict() # log average average particle count list
log_avg_count_lat = dict() # log average box size list

for lat in lat_size:
    log_box_count_lat[lat] = dict()
    log_avg_count_lat[lat] = dict()
    log_box_sizes_lat[lat] = dict()
    for particles in simulations[lat]:
        log_box_count_lat[lat][particles] = dict()
        log_avg_count_lat[lat][particles] = dict()
        log_box_sizes_lat[lat][particles] = dict()
        for probability in simulation_probabilities[lat]:
            try:
                log_box_count_lat[lat][particles][probability] = np.log(box_count_lat_file[lat][particles][probability][0])
                log_avg_count_lat[lat][particles][probability] = np.log(avg_count_lat_file[lat][particles][probability][0])
                log_box_sizes_lat[lat][particles][probability] = np.log(box_sizes_lat_file[lat][particles][probability][0])
            except:
                continue

print(log_box_count_lat)
print(log_avg_count_lat)
print(log_box_sizes_lat)

In [ ]:
# Linearization
linear_box_count_lat = dict()
for lat in lat_size:
    linear_box_count_lat[lat] = dict()
    for particles in simulations[lat]:
        if particles in log_box_count_lat[lat]:
            linear_box_count_lat[lat][particles] = dict()
        for probability in simulation_probabilities[lat]:
            if probability in log_box_count_lat[lat][particles]:
                linear_box_count_lat[lat][particles][probability] = dict()
                linear_box_count_lat[lat][particles][probability]['pcount'] = linearReg(log_box_sizes_lat[lat][particles][probability], log_avg_count_lat[lat][particles][probability])
                linear_box_count_lat[lat][particles][probability]['bcount'] = linearReg(log_box_sizes_lat[lat][particles][probability], log_box_count_lat[lat][particles][probability])
                
print(linear_box_count_lat)

In [ ]:
#Df for same density
particular_density = (8384 * np.pi/4)/(512**2)
particular_lat = [128, 256, 512]
particular_part = [524, 2098, 8384]
particular_probs = [0.01, 0.015]
labels = []
avg_fractal_dim = []

plt.figure(figsize=(12,9))
for lat, c in zip(particular_lat, colors):
    for part in particular_part:
        for prob, m in zip(particular_probs, markers):
            try:
                plt.scatter(lat, (linear_box_count_lat[lat][part][prob]['pcount'][0] - linear_box_count_lat[lat][part][prob]['bcount'][0])/ 2, color=c, marker=m, s=50, label = f"Particles: {part}, Probability: {prob}")
                avg_fractal_dim.append((linear_box_count_lat[lat][part][prob]['pcount'][0] - linear_box_count_lat[lat][part][prob]['bcount'][0]) / 2)
                labels.append(f"Lat Size: {lat}, \nParticles: {part},\n Probability: {prob}")
            except:
                continue
plt.title(f"Average Fractal Dimensios for Different Systems with Density {particular_density:.4f}", fontsize = 16)
plt.xlabel("Lattice Size", fontsize=16)
plt.ylabel("Fractal Dimension", fontsize=16)
plt.xticks(fontsize = 16)
plt.yticks(fontsize = 16)
plt.ylim(0,1.5)
plt.legend(fontsize=16)
plt.savefig("../Imagenes/DimensionFractalvsDensidadCnst.png")


plt.figure(figsize=(12,9))
plt.title(f"Average Fractal Dimensios for Different Systems with Density {particular_density:.4f}", fontsize = 16)
plt.bar(labels, avg_fractal_dim, color="darkblue")
plt.ylabel("Fractal Dimension", fontsize = 16)
plt.xticks(fontsize = 16, rotation=90)
plt.yticks(fontsize = 16)
plt.savefig("../Imagenes/DimensionFractalvsDensidadBAR.png")
plt.show()


In [ ]:
# Df Vs. Prob
handles = []
plt.figure(figsize=(12,9))
for lat, c in zip([512], colors[1:]):
    for part, m in zip(simulations[lat], markers):
        handles.append(Line2D([], [], color=c, marker=m, linestyle='None',
                          markersize=10, label=f"Lat Size: {lat}, Particles: {part}"))
        probs = []
        frac_dims = []
        for prob in simulation_probabilities[lat]:
            try:
                frac_dims.append((linear_box_count_lat[lat][part][prob]['pcount'][0] - linear_box_count_lat[lat][part][prob]['bcount'][0])/ 2)
                probs.append(prob)
            except:
                continue
        plt.plot(probs, frac_dims, color=c, marker=m, markersize=10)
plt.title(f"Average Fractal Dimensios for Different Probabilities", fontsize = 16)
plt.xlabel("Probability", fontsize=16)
plt.ylabel("Fractal Dimension", fontsize=16)
plt.xticks(fontsize = 16)
plt.yticks(fontsize = 16)
# plt.ylim(0,1.5)
plt.legend(handles=handles, fontsize=16)
plt.savefig("../Imagenes/DimensionFractalvsProb.png")
plt.show()

In [ ]:
#Df varying Density

handles = []
plt.figure(figsize=(12,9))
for lat in [512]:
    for part, c  in zip(simulations[lat], colors):
        densities = []
        frac_dims = []
        for prob, m in zip(simulation_probabilities[lat], markers):
            try:
                frac_dims.append((linear_box_count_lat[lat][part][prob]['pcount'][0] - linear_box_count_lat[lat][part][prob]['bcount'][0])/ 2)
                densities.append((part * (np.pi/4)) / (lat ** 2))
                handles.append(Line2D([], [], color=c, marker=m, linestyle='None',
                          markersize=10, label=f"Lat Size: {lat}, Particles: {part}, Probability: {prob}"))
                plt.scatter(densities, frac_dims, color=c, marker=m, s=50)
            except:
                continue
        
plt.title(f"Average Fractal Dimensios for Different Probabilities", fontsize = 16)
plt.xlabel("Density", fontsize=16)
plt.ylabel("Fractal Dimension", fontsize=16)
plt.xticks(fontsize = 16)
plt.yticks(fontsize = 16)
plt.ylim(0,1.6)
plt.legend(handles=handles, fontsize=16)
plt.savefig("../Imagenes/DimensionFractalvsDens.png")
plt.show()   

### Final Coordination Numbers

In [ ]:
def coordinationNumbers(lat_size, particles, prob):
    zs = []
    mean = df[(df['num_particles'] == particles) & (df['lattice_size'] == lat) & (df['probability'] == prob)].mean()[6:]  #Change slice to adequate value
    for i, val in enumerate(mean):
            zs.append(mean[i] / particles)
    return zs

plt.figure(figsize = (9, 9))
legend_elements = []

custom_labels = []
for lat, marker in zip(lat_size, markers):
    for particles in [simulations[lat][0]]:
        custom_labels.append(Line2D([0], [0], color='k', marker=marker, markersize=10, label=f"Lat Size: {lat}, Particles = {particles}"))
        zs_total = []
        for prob in simulation_probabilities[lat]:
            zs = coordinationNumbers(lat, particles, prob)
            if zs:
                zs_total.append(zs)

        if zs_total:
            for j, zs in enumerate(zip(*zs_total)):
                plt.plot(simulation_probabilities[lat], zs, color = colors[j], marker=marker,markersize=10)

for i in range(7):
    custom_labels.append(Line2D([0], [0], color=colors[i], label=f"z = {i}"))

plt.title(f"Mean Coordination Number vs Probability of Separation for {simulation_densities[512][0]:.3f} Density", fontsize = 16)
plt.xlabel("Probability", fontsize = 16)
plt.ylabel("Percentage of Particles", fontsize = 16)
plt.xticks(fontsize = 16)
plt.yticks(fontsize = 16)
plt.legend(handles=custom_labels, fontsize = 16, loc = (1.0, 0.25))
# plt.savefig("Result Images/ResultsZ_Off.png")
plt.show()

In [ ]:
plt.figure(figsize = (9, 9))
linestyles = ['-', '--', '-.', ':', "solid"]
legend_elements = []

custom_labels = []
lat = 512
for density, particles, marker, ls in zip(simulation_densities[lat], simulations[lat], markers, linestyles):
    custom_labels.append(Line2D([0], [0], color='k', linestyle=ls, marker=marker, markersize=10, label=f"Particles = {particles}"))
    zs_total = []
    for prob in simulation_probabilities[lat]:
        zs = coordinationNumbers(lat, particles, prob)
        if zs:
            zs_total.append(zs)

    if zs_total:
        for j, zs in enumerate(zip(*zs_total)):
            plt.plot(simulation_probabilities[lat], zs, color = colors[j], marker=marker,markersize=10, linestyle=ls)

for i in range(7):
    custom_labels.append(Line2D([0], [0], color=colors[i], label=f"z = {i}"))

plt.title(f"Mean Coordination Number vs Probability Separation for Lat Size = {lat} for Different Densities", fontsize = 16)
plt.xlabel("Probability", fontsize = 16)
plt.ylabel("Percentage of Particles", fontsize = 16)
plt.xticks(fontsize = 14)
plt.yticks(fontsize = 16)
plt.legend(handles=custom_labels, fontsize = 16, loc = (1.0, 0.25))
plt.savefig("../Imagenes/ResultsZ.png")
plt.show()

### Avg. Cluster Mass Per Step

In [ ]:
def massStepDict(lat_size, particles, probability):
    avg_mass = dict()

    mass_time_files = [f for f in os.listdir(path_seagate) if (mass_time_fn in f) and (str(int(lat_size)) in f) and (str(int(particles)) in f) and (f"{probability:.3f}" in f) and not (edges_fn in f)]
    if (len(mass_time_files) == 0):
        return dict()

    for file_name in mass_time_files:
        try:
            data = np.loadtxt(path_seagate + file_name, delimiter=',')
            for line in data:
                if line[0] in avg_mass:
                    avg_mass[line[0]][0] = avg_mass[line[0]][0] + 1
                    avg_mass[line[0]][1].append(particles/np.sum(line[1:]))
                else:
                    avg_mass[line[0]] = [1, [particles/np.sum(line[1:])]]
        except:
            continue
    

    return dict(sorted(avg_mass.items()))

In [ ]:
avg_mass_dicts = []
avg_mass_total_mean = []
for lat in [512]:
    for particles in simulations[lat]:
        for prob in simulation_probabilities[lat]:
            avg_mass = massStepDict(lat, particles, prob)
            if avg_mass:
                avg_mass_total = []
                plt.figure(figsize = (15,9))
                for step_num in avg_mass:
                    plt.scatter([step_num]*len(avg_mass[step_num][1]), avg_mass[step_num][1], color=colors[1],alpha=0.2)
                    avg_mass_total.append(np.mean(avg_mass[step_num][1]))

                avg_mass_dicts.append(avg_mass)
                print(f"Plotting L = {lat}, Particles = {particles}, and Prob = {prob}")
                plt.title(f"Average Cluster Mass for Every Step Taken L = {lat}, Particles = {particles} and Probability = {prob}")
                plt.xlabel("Num Steps")
                plt.ylabel("Avg Mass")
                plt.plot(avg_mass.keys(), avg_mass_total, marker = "o")
                plt.savefig(f"../Imagenes/AverageMass{lat}Particles{particles}Prob{prob}.png")
                avg_mass_total_mean.append(avg_mass_total)
plt.show()

In [ ]:
# Softening previous curves
titles = [f"Average Cluster Mass for Every Step Taken L = {lat}, Particles = {particles} and Probability = {prob}" for lat in [512] for particles in simulations[lat] for prob in simulation_probabilities[lat] if len(df[(df['num_particles'] == particles) & (df['lattice_size'] == lat) & (df['probability'] == prob)]) != 0]

particles_plot = [particles for lat in [512] for particles in simulations[lat] for prob in simulation_probabilities[lat] if len(df[(df['num_particles'] == particles) & (df['lattice_size'] == lat) & (df['probability'] == prob)]) != 0]
prob_plot = [prob for lat in [512] for particles in simulations[lat] for prob in simulation_probabilities[lat] if len(df[(df['num_particles'] == particles) & (df['lattice_size'] == lat) & (df['probability'] == prob)]) != 0]


steps_soft_all_avg = []
avg_avg_soft_all = []
for i, (avg_mass, avg_mass_total) in enumerate(zip(avg_mass_dicts, avg_mass_total_mean)):
    steps_soft = soften(list(avg_mass.keys()), len(avg_mass.keys())//50)
    mass_soft = soften(avg_mass_total, len(avg_mass_total)//50)
    if avg_mass:
        plt.figure(figsize = (15,9))
        for step_num in avg_mass:
                plt.scatter([step_num]*len(avg_mass[step_num][1]), avg_mass[step_num][1], color=colors[1],alpha=0.2)
        plt.title(titles[i])
        plt.xlabel("Num Steps")
        plt.ylabel("Size of Clusters")
        plt.plot(steps_soft[:-1], mass_soft[:-1], marker = "o") 

        plt.savefig(f"../Imagenes/SoftAverageMass{512}Particles{particles_plot[i]}Prob{prob_plot[i]}.png") 

        steps_soft_all_avg.append(steps_soft)
        avg_avg_soft_all.append(mass_soft)

plt.show()

### Number of Clusters vs Steps

In [ ]:
def numClustersStepDict(lat_size, particles, probability):
    avg_mass = dict()

    mass_time_files = [f for f in os.listdir(path_seagate) if (mass_time_fn in f) and (str(int(lat_size)) in f) and (str(int(particles)) in f) and (f"{probability:.3f}" in f) and not (edges_fn in f)]
    if (len(mass_time_files) == 0):
        return dict()

    for file_name in mass_time_files:
        try:
            data = np.loadtxt(path_seagate + file_name, delimiter=',')
            for line in data:
                if line[0] in avg_mass:
                    avg_mass[line[0]][0] = avg_mass[line[0]][0] + 1
                    avg_mass[line[0]][1].append(np.sum(line[1:]))
                else:
                    avg_mass[line[0]] = [1, [np.sum(line[1:])]]
        except:
            continue
    

    return dict(sorted(avg_mass.items()))


num_clusters_dicts = []
num_clusters_total_mean = []
for lat in [512]:
    for particles in simulations[lat]:
        for prob in simulation_probabilities[lat]:
            avg_mass = numClustersStepDict(lat, particles, prob)
            if avg_mass:
                avg_mass_total = []
                plt.figure(figsize = (15,9))
                for step_num in avg_mass:
                    plt.scatter([step_num]*len(avg_mass[step_num][1]), avg_mass[step_num][1], color=colors[1],alpha=0.2)
                    avg_mass_total.append(np.mean(avg_mass[step_num][1]))

                num_clusters_dicts.append(avg_mass)
                print(f"Plotting L = {lat}, Particles = {particles}, and Prob = {prob}")
                plt.title(f"Average Number of Clusters for Every Step Taken L = {lat}, Particles = {particles} and Probability = {prob}")
                plt.xlabel("Num Steps")
                plt.ylabel("Num. Clusters")
                plt.plot(avg_mass.keys(), avg_mass_total, marker = "o")
                plt.savefig(f"../Imagenes/NumClustersSize{lat}Particles{particles}Prob{prob}.png")
                num_clusters_total_mean.append(avg_mass_total)
plt.show()

### Largest Cluster Size vs Steps

In [ ]:
def getLargestFilledIndex(array):
    i = -1
    for j, val in enumerate(array):
        if val != 0:
            i = j+1

    return i

def largestClustersStepDict(lat_size, particles, probability):
    avg_mass = dict()

    mass_time_files = [f for f in os.listdir(path_seagate) if (mass_time_fn in f) and (str(int(lat_size)) in f) and (str(int(particles)) in f) and (f"{probability:.3f}" in f) and not (edges_fn in f)]
    if not mass_time_files:
        return dict()

    for file_name in mass_time_files:
        try:
            data = np.loadtxt(path_seagate + file_name, delimiter=',')
            for line in data:
                if line[0] in avg_mass:
                    avg_mass[line[0]][0] = avg_mass[line[0]][0] + 1
                    avg_mass[line[0]][1].append(getLargestFilledIndex(line[1:]))
                else:
                    avg_mass[line[0]] = [1, [getLargestFilledIndex(line[1:])]]
        except: 
            continue    

    return dict(sorted(avg_mass.items()))


avg_largest_dicts = []
avg_largest_mean = []
for lat in lat_size:
    for particles in simulations[lat]:
        for prob in simulation_probabilities[lat]:
            avg_mass = largestClustersStepDict(lat, particles, prob)
            if avg_mass:
                avg_largest = []
                plt.figure(figsize = (15,9))
                for step_num in avg_mass:
                    plt.scatter([step_num]*len(avg_mass[step_num][1]), avg_mass[step_num][1], color=colors[1],alpha=0.2)
                    avg_largest.append(np.mean(avg_mass[step_num][1]))

                avg_largest_dicts.append(avg_mass)
                print(f"Plotting L = {lat}, Particles = {particles}, and Prob = {prob}")
                plt.title(f"Largest Cluster Size for Every Step Taken L = {lat}, Particles = {particles} and Probability = {prob}")
                plt.xlabel("Num Steps")
                plt.ylabel("Size of Clusters")
                plt.plot(avg_mass.keys(), avg_largest, marker = "o")
                plt.savefig(f"../Imagenes/LargestCluster{lat}Particles{particles}Prob{prob}.png")
                avg_largest_mean.append(avg_largest)
plt.show()

In [ ]:
# Softening previous curves
titles = [f"Largest Cluster Size for Every Step Taken L = {lat}, Particles = {particles} and Probability = {prob}" for lat in lat_size for particles in simulations[lat] for prob in simulation_probabilities[lat] if len(df[(df['num_particles'] == particles) & (df['lattice_size'] == lat) & (df['probability'] == prob)]) != 0]


particles_plot = [particles for lat in lat_size for particles in simulations[lat] for prob in simulation_probabilities[lat] if len(df[(df['num_particles'] == particles) & (df['lattice_size'] == lat) & (df['probability'] == prob)]) != 0]
prob_plot = [prob for lat in lat_size for particles in simulations[lat] for prob in simulation_probabilities[lat] if len(df[(df['num_particles'] == particles) & (df['lattice_size'] == lat) & (df['probability'] == prob)]) != 0]


steps_soft_all_largest = []
avg_largest_soft_all = []
for i, (avg_mass, avg_largest) in enumerate(zip(avg_largest_dicts, avg_largest_mean)):
    steps_soft = soften(list(avg_mass.keys()), len(avg_mass.keys())//50)
    mass_soft = soften(avg_largest, len(avg_largest)//50)
    if avg_mass:
        plt.figure(figsize = (15,9))
        # for step_num in avg_mass:
                # plt.scatter([step_num]*len(avg_mass[step_num][1]), avg_mass[step_num][1], color=colors[1],alpha=0.2)
        plt.title(titles[i])
        plt.xlabel("Num Steps")
        plt.ylabel("Size of Clusters")
        plt.plot(steps_soft[:-1], mass_soft[:-1], marker = "o")  
        plt.savefig(f"../Imagenes/SoftLargestMass{512}Particles{particles_plot[i]}Prob{prob_plot[i]}.png") 

        steps_soft_all_largest.append(steps_soft)
        avg_largest_soft_all.append(mass_soft)

plt.show()  

In [ ]:
# Varying probability and plotting same densities together
particular_lat = 512
i = 14
for particles in simulations[particular_lat]:
    plt.figure(figsize=(15,9))
    for prob, c in zip(simulation_probabilities[particular_lat], colors):
        plt.plot(steps_soft_all_largest[i][:-1], avg_largest_soft_all[i][:-1], color=c, label=f"Particles = {particles}\nProbability = {prob}")
        plt.title(f"Largest Cluster for Every Step Taken for L = {particular_lat} with Density = {(particles * (np.pi/4)) / (particular_lat**2):.4f}")
        plt.xlabel("Number of Steps")
        plt.ylabel("Mass of Largest Cluster")
        plt.legend()
        plt.savefig(f"../Imagenes/LargestClusterVaryingL{particular_lat}Particles{particles}")
        
        i += 1



In [ ]:
# Varying probability and plotting same densities together
particular_lat = 512
i = 14
for particles in simulations[particular_lat]:
    plt.figure(figsize=(15,9))
    for prob, c in zip(simulation_probabilities[particular_lat], colors):
        if (i < 22 and prob != 0.01) or (i >=22 and prob != 0.015 and prob != 0.01):
            plt.plot(steps_soft_all_largest[i][:-1], avg_largest_soft_all[i][:-1], color=c, label=f"Probability = {prob}")
            plt.title(f"Largest Cluster for Every Step Taken for L = {particular_lat}, Particles = {particles}, and Density = {(particles * (np.pi/4)) / (particular_lat**2):.4f}")
            plt.xlabel("Number of Steps")
            plt.ylabel("Mass of Largest Cluster")
            plt.legend()

            plt.savefig(f"../Imagenes/ReducedLargestClusterVaryingL{particular_lat}Particles{particles}")

        i += 1



In [ ]:
# Varying density and plotting same porbabilities together
particular_lat = 512
plots = []
figs = []
for i in range(4):
    plot = plt.figure(figsize=(15,9))
    ax = plot.add_subplot(1,1,1)
    plots.append(ax)
    figs.append(plot)

cont = 14
i = 0
for particles, c in zip(simulations[particular_lat], colors):
    j = 0
    for prob in simulation_probabilities[particular_lat]:
        if i == 0:
            plots[j].set_title(f"Largest Cluster for Every Step Taken for L = {particular_lat} and Probability = {prob}")
            plots[j].set_xlabel("Number of Steps")
            plots[j].set_ylabel("Normalized Mass of Largest Cluster")
        
        plots[j].plot(steps_soft_all_largest[cont][:-1], np.array(avg_largest_soft_all[cont][:-1])/particles , color=c, label=f"Particles = {particles}")

        j += 1
        cont += 1

    i += 1

for i in range(4):
    plots[i].legend()
    figs[i].savefig(f"../Imagenes/LargestClusterVaryingL{particular_lat}Probability{simulation_probabilities[particular_lat][i]}.png")




### Percolation

In [ ]:
perc_percolation = []
labels = []
percolation_colors = []
for lat in [512]:
    for particles in simulations[lat]:
        for i, prob in enumerate(simulation_probabilities[lat]):
            if not ((particles == 8384 and prob == 0.015) or (particles == 12582 and prob == 0.02) and (particles == 14680 and prob == 0.02)):
                if (len(df[(df.num_particles == particles) & (df.probability == prob) & (df.lattice_size == lat)]) != 0):
                    perc_percolation.append(df[(df.num_particles == particles) & (df.probability == prob) & (df.lattice_size == lat)].percolation.mean())
                    labels.append(f"L = {lat}, P = {particles} & Prob = {prob}")
                    percolation_colors.append(colors[i+1])

print(perc_percolation)
plt.figure(figsize = (15,9))
plt.bar(labels, perc_percolation, color = percolation_colors)
plt.title("Percentage of Particles that Percolate vs. Simulated Clusters", fontsize = 16)
plt.xlabel("Probability and Size", fontsize = 16)
plt.ylabel("Percentage of Particles", fontsize = 16)
plt.xticks(fontsize = 16, rotation=90)
plt.yticks(fontsize = 16)
plt.grid(axis='y')
plt.savefig(f"../Imagenes/PercolateOffRevSimplified.png")
plt.show()


### Gráficas simulaciones largas

In [ ]:
from csv import reader
def getLargestFilledIndex(array):
    i = -1
    for j, val in enumerate(array):
        if val != 0:
            i = j+1

    return i


def longSims(file_name):
    steps = []
    largest = []
    with open(file_name, 'r') as f:
        csv_reader = reader(f)
        for row in csv_reader:
            steps.append(int(row[0]))
            largest.append(getLargestFilledIndex(row[1:]))

    return steps, largest

file_name = ["Results/NumberTimeSize512Particles10486Prob0.020000.csv", "Results/NumberTimeSize512Particles15728Prob0.030000.csv"]
titles = ["Largest Cluster for Every Step Taken L = 512, Particles = 10486 and Probability = 0.02",
            "Largest Cluster for Every Step Taken L = 512, Particles = 15728 and Probability = 0.03"]
for f, title in zip(file_name, titles):
    print(f"Getting x and y values for {title}")
    x, y = longSims(f)

    print("Softening")
    print(len(x), len(y))
    x_soft = soften(x, len(x)//1000)
    y_soft = soften(y, len(y)//1000)
    print("Plotting")
    plt.figure(figsize=(12,9))
    plt.plot(x_soft, y_soft, marker='o')
    plt.xlabel("Num Steps")
    plt.ylabel("Largest Cluster")

plt.show()


# Probability Cluster

In [ ]:
df_edges = pd.read_csv("Final Results/ResultsOffLEdge.csv")
lat_size_edges = df_edges.lattice_size.unique()
lat_size_edges.sort()
print(lat_size_edges)

radius = 1

simulation_edges = dict()
simulation_probabilities_edges = dict()
simulation_densities_edges = dict()
for lat in lat_size_edges:
    simulation_edges[lat] = sorted(list(df_edges[df_edges.lattice_size == lat].num_particles.unique()))
    simulation_probabilities_edges[lat] = sorted(list(df_edges[df_edges.lattice_size == lat].probability.unique()))
    simulation_densities_edges[lat] = [(particles * np.pi * (radius ** 2)) / (4 * (lat ** 2)) for particles in sorted(list(df_edges[df_edges.lattice_size == lat].num_particles.unique()))]

print(simulation_edges)
print(simulation_probabilities_edges)
print(simulation_densities_edges)

In [ ]:
box_count_lat_edges = dict()
avg_count_lat_edges = dict()
box_sizes_lat_edges = dict()

for lat in lat_size_edges:
    box_count_lat_edges[lat] = dict()
    avg_count_lat_edges[lat] = dict()
    box_sizes_lat_edges[lat] = dict()
    for particles in simulation_edges[lat]:
        box_count_lat_edges[lat][particles] = dict()
        avg_count_lat_edges[lat][particles] = dict()
        box_sizes_lat_edges[lat][particles] = dict()
        for probability in simulation_probabilities_edges[lat]:
            box_counts_list = []
            avg_counts_list = []
            box_sizes_list = []

            text_files = [f for f in os.listdir(path_seagate) if (cluster_fn in f) and (str(int(lat)) in f) and (str(int(particles)) in f) and (str(float(probability)) in f) and (edges_fn in f)]
            if text_files:
                box_count_lat_edges[lat][particles][probability] = []
                avg_count_lat_edges[lat][particles][probability] = []
                box_sizes_lat_edges[lat][particles][probability] = []
                print(f"Computing Fractal Dims for Lat = {lat}, Particles = {particles} and Probability = {probability} Edges")
                for file_i in text_files:
                    cluster = np.loadtxt(path_seagate + '/' + file_i, delimiter = ',')

                    x, y, index, z = np.hsplit(cluster, cluster.shape[1])
                    x = x.flatten()
                    y = y.flatten()
                    indexes = index.flatten().astype(int)
                    largest_cluster = int(stats.mode(indexes).mode)

                    x_index = x[np.where(indexes == largest_cluster)]
                    y_index = y[np.where(indexes == largest_cluster)]
            
                    if len(x_index) > 500:
                        with blockPrint():
                            box_count, avg_counts, box_size = boxCount(lat, len(x_index), x_index, y_index)
                        plt.close('all')

                        box_counts_list.append(box_count)
                        avg_counts_list.append(avg_counts)
                        box_sizes_list.append(box_size)

            if box_counts_list:
                box_count_avg_p = np.mean(box_counts_list, axis = 0)
                avg_count_avg_p = np.mean(avg_counts_list, axis = 0)
                box_sizes_avg_p = np.mean(box_sizes_list, axis = 0)
                box_count_lat_edges[lat][particles][probability].append(box_count_avg_p)
                avg_count_lat_edges[lat][particles][probability].append(avg_count_avg_p)
                box_sizes_lat_edges[lat][particles][probability].append(box_sizes_avg_p)

print(box_count_lat_edges)

print("Computing logarithm of Lists")
log_box_sizes_lat_edges = dict() # log average box count list
log_box_count_lat_edges = dict() # log average average particle count list
log_avg_count_lat_edges = dict() # log average box size list

for lat in lat_size_edges:
    log_box_count_lat_edges[lat] = dict()
    log_avg_count_lat_edges[lat] = dict()
    log_box_sizes_lat_edges[lat] = dict()
    for particles in simulation_edges[lat]:
        log_box_count_lat_edges[lat][particles] = dict()
        log_avg_count_lat_edges[lat][particles] = dict()
        log_box_sizes_lat_edges[lat][particles] = dict()
        for probability in simulation_probabilities_edges[lat]:
            try:
                log_box_count_lat_edges[lat][particles][probability] = np.log(box_count_lat_edges[lat][particles][probability][0])
                log_avg_count_lat_edges[lat][particles][probability] = np.log(avg_count_lat_edges[lat][particles][probability][0])
                log_box_sizes_lat_edges[lat][particles][probability] = np.log(box_sizes_lat_edges[lat][particles][probability][0])
            except:
                continue

print("Logs completed")
print(log_box_sizes_lat_edges)
# Linearization
linear_box_count_lat_edges = dict()
for lat in lat_size_edges:
    linear_box_count_lat_edges[lat] = dict()
    for particles in simulation_edges[lat]:
        if particles in log_box_count_lat_edges[lat]:
            linear_box_count_lat_edges[lat][particles] = dict()
        for probability in simulation_probabilities_edges[lat]:
            if probability in log_box_count_lat_edges[lat][particles]:
                linear_box_count_lat_edges[lat][particles][probability] = dict()
                linear_box_count_lat_edges[lat][particles][probability]['pcount'] = linearReg(log_box_sizes_lat_edges[lat][particles][probability], log_avg_count_lat_edges[lat][particles][probability])
                linear_box_count_lat_edges[lat][particles][probability]['bcount'] = linearReg(log_box_sizes_lat_edges[lat][particles][probability], log_box_count_lat_edges[lat][particles][probability])

print("Linearization Complete")
print(linear_box_count_lat_edges)

In [ ]:
labels = []
avg_fractal_dim = []

plt.figure(figsize=(12,9))
for lat, c in zip(lat_size_edges, colors):
    for part in simulation_edges[lat]:
        for prob, m in zip(simulation_probabilities_edges[lat], markers):
            try:
                plt.scatter(part, (linear_box_count_lat_edges[lat][part][prob]['pcount'][0] - linear_box_count_lat_edges[lat][part][prob]['bcount'][0])/ 2, color=c, marker=m, s=50, label = f"Lat Size: {lat}, Probability: {prob}")
                
                avg_fractal_dim.append((linear_box_count_lat_edges[lat][part][prob]['pcount'][0] - linear_box_count_lat_edges[lat][part][prob]['bcount'][0]) / 2)
                labels.append(f"Lat Size: {lat}, \nParticles: {part},\n Probability: {prob}")
            except:
                continue

plt.title(f"Average Fractal Dimensios for Different Systems with Density {simulation_densities_edges[512][0]:.4f}", fontsize = 16)
plt.xlabel("Lattice Size", fontsize=16)
plt.ylabel("Fractal Dimension", fontsize=16)
plt.xticks(fontsize = 16)
plt.yticks(fontsize = 16)
plt.ylim(0,1.5)
plt.savefig(f"../Imagenes/AverageFractalDimEdgesRev")
plt.legend(fontsize=16)


plt.figure(figsize=(9,9))
plt.title(f"Average Fractal Dimensios for Different Systems with Density {simulation_densities_edges[512][1]:.4f}", fontsize = 16)
plt.bar(labels, avg_fractal_dim, color="darkblue")
plt.ylabel("Fractal Dimension", fontsize = 16)
plt.xticks(fontsize = 16, rotation=90)
plt.yticks(fontsize = 16)
# plt.ylim(0,1.5)
plt.show()

In [ ]:
def getLargestFilledIndex(array):
    i = -1
    for j, val in enumerate(array):
        if val != 0:
            i = j+1

    return i

def largestClustersStepDictEdges(lat_size, particles, probability):
    avg_mass = dict()

    mass_time_files = [f for f in os.listdir(path_seagate) if (mass_time_fn in f) and (str(int(lat_size)) in f) and (str(int(particles)) in f) and (f"{probability:.3f}" in f) and (edges_fn in f)]
    if not mass_time_files:
        return dict()

    for file_name in mass_time_files:
        try:
            data = np.loadtxt(path_seagate + file_name, delimiter=',')
            for line in data:
                if line[0] in avg_mass:
                    avg_mass[line[0]][0] = avg_mass[line[0]][0] + 1
                    avg_mass[line[0]][1].append(getLargestFilledIndex(line[1:]))
                else:
                    avg_mass[line[0]] = [1, [getLargestFilledIndex(line[1:])]]
        except: 
            continue    

    return dict(sorted(avg_mass.items()))

avg_largest_dicts_edges = []
mean_plot_averages_edge = []
for lat in lat_size_edges:
    for particles in simulation_edges[lat]:
        for prob in simulation_probabilities_edges[lat]:
            avg_mass = largestClustersStepDictEdges(lat, particles, prob)
            if avg_mass:
                mean_plot = []
                plt.figure(figsize = (15,9))
                for step_num in avg_mass:
                    plt.scatter([step_num]*len(avg_mass[step_num][1]), avg_mass[step_num][1], color=colors[1],alpha=0.2)
                    mean_plot.append(np.mean(avg_mass[step_num][1]))
                avg_largest_dicts_edges.append(avg_mass)
                print(f"Plotting L = {lat}, Particles = {particles}, and Prob = {prob}")
                plt.title(f"Largest Cluster Size for Every Step Taken L = {lat}, Particles = {particles} and Probability = {prob}")
                plt.xlabel("Num Steps")
                plt.ylabel("Size of Clusters")
                plt.plot(avg_mass.keys(), mean_plot, marker = "o")
                mean_plot_averages_edge.append(mean_plot)
plt.show()

In [ ]:
# Softening previous curves
titles = [f"Largest Edge Cluster Size for Every Step Taken L = {lat}, Particles = {particles} and Probability = {prob}" for lat in [512] for particles in simulations_edges[lat] for prob in simulation_probabilities_edges[lat] if len(df_edges[(df_edges['num_particles'] == particles) & (df_edges['lattice_size'] == lat) & (df_edges['probability'] == prob)]) != 0]


particles_plot = [particles for lat in [512] for particles in simulations_edges[lat] for prob in simulation_probabilities_edges[lat] if len(df_edges[(df_edges['num_particles'] == particles) & (df_edges['lattice_size'] == lat) & (df_edges['probability'] == prob)]) != 0]
prob_plot = [prob for lat in [512] for particles in simulations_edges[lat] for prob in simulation_probabilities_edges[lat] if len(df_edges[(df_edges['num_particles'] == particles) & (df_edges['lattice_size'] == lat) & (df_edges['probability'] == prob)]) != 0]


steps_soft_all_largest = []
avg_largest_soft_all = []
for i, (avg_mass, avg_largest) in enumerate(zip(avg_largest_dicts_edges, avg_largest_mean_edge)):
    steps_soft = soften(list(avg_mass.keys()), len(avg_mass.keys())//50)
    mass_soft = soften(avg_largest, len(avg_largest)//50)
    if avg_mass:
        plt.figure(figsize = (15,9))
        for step_num in avg_mass:
                plt.scatter([step_num]*len(avg_mass[step_num][1]), avg_mass[step_num][1], color=colors[1],alpha=0.2)
        plt.title(titles[i])
        plt.xlabel("Num Steps")
        plt.ylabel("Size of Clusters")
        plt.plot(steps_soft[:-1], mass_soft[:-1], marker = "o")  
        plt.savefig(f"../Imagenes/SoftAverageMass{512}Particles{particles_plot[i]}Prob{prob_plot[i]}Edge.png") 

        steps_soft_all_largest.append(steps_soft)
        avg_largest_soft_all.append(mass_soft)

plt.show()  